# Deploying your trained model

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()

In [ ]:
from azureml.core.model import Model

model_name = "MY_MODEL_NAME"
service_name = "MY_WEBSERVICE_NAME"

# Fetch model from workspace. Include version parameter
# to fetch a specific version. 
model = Model(workspace=workspace, name=model_name)

In [ ]:
from pathlib import Path

# Define model environment.
model_dir = Path("../model")
model_env = Environment.from_conda_specification(f"{model_name}-env", model_dir / "environment.yml")
 
# Define inference config (= how to serve model).
inference_config = InferenceConfig(
    entry_script="scripts/serve.py", 
    source_directory=model_dir, 
    environment=env
)

# Remove any existing service under the same name.
service_name = "titanic"

try:
    existing_service = Webservice(workspace, name=service_name)
    print("WARNING: Deleting existing service '%s'", service_name)
    existing_service.delete()
except WebserviceException:
    pass

print("Starting deployment")
service = Model.deploy(workspace, service_name, [model], inference_config)

if wait:
    service.wait_for_deployment(show_output=True)

### Exercise 

Try to query your webservice using the requests library.

In [ ]:
%load answers/webservice.py